In [1]:
import pickle
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Conv1D, InputLayer, Masking, MaxPooling1D, GlobalAveragePooling1D, Dropout

In [2]:
class BaseModel:
    def __init__(self, train):
        self.inputs, self.ys = self.get_train(train)
        
        self.model = self.build_model()
        
        self.callbacks = [
            keras.callbacks.ModelCheckpoint(
                filepath='models.{epoch:02d}-{val_loss:.2f}.h5',
                save_best_only=True
            )
        ]
        
    def get_train(self, train):
        inputs = []
        ys = []
        
        for t in train:
            sig = t['sig']
            len_sig = t['sig_len']
            fs = t['fs']
            for i in range(int(len_sig/fs)):
                # input range: sig[i:i+fs]
                if i+fs <= len(sig):
                    input_sig = sig[i:i+fs]
                else:
                    input_sig = sig[i:]
                inputs.append(input_sig)
                
                if t['class_true'] == 0:
                    y = 0
                elif t['class_true'] == 1:
                    y = 1
                else:
                    y = 0
                    for j in range(len(t['af_start_scripts'])):
                        if i in range(t['af_start_scripts'][j], t['af_end_scripts'][j] + 1):
                            y = 1
                            break
                    
                ys.append(y)
                
        inputs = np.array(inputs)
        ys = np.array(ys)
        
        return inputs, ys
    
    def build_model(self):
        model = Sequential()
        model.add(Masking(mask_value=0, input_shape=(200, 2)))
        model.add(Conv1D(16, 12, activation='relu'))
        model.add(Conv1D(16, 12, activation='relu'))
        model.add(MaxPooling1D(3))
        model.add(Conv1D(32, 12, activation='relu'))
        model.add(Conv1D(32, 12, activation='relu'))
        model.add(GlobalAveragePooling1D())
#         model.add(Dropout(0.5))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(16, activation='relu'))
        model.add(Dense(3, activation='softmax')) # af start, af end, non af
        
        return model

    
    def fit(self):
        self.model.compile(loss='categorical_crossentropy',
                           optimizer='adam',
                           metrics=['accuracy'])
        
        self.model.summary()
                
        self.model.fit(self.inputs,
                       to_categorical(self.ys, 3),
                       batch_size = 500,
                       epochs = 5,
                       verbose = 1,
                       validation_split = 0.2,
                       callbacks = self.callbacks)
        
    def predict(self):
        pass

In [3]:
TRAINING_DATA_PATH = '../data/train.pkl'
TEST_DATA_PATH = '../data/test.pkl'

# load data
with open(TRAINING_DATA_PATH, 'rb') as file:
    train = pickle.load(file)

with open(TEST_DATA_PATH, 'rb') as file:
    test = pickle.load(file)

In [4]:
train[0].keys()

dict_keys(['record_name', 'sig', 'sig_len', 'fs', 'beat_loc', 'af_start_scripts', 'af_end_scripts', 'class_true'])

In [5]:
# for i, t in enumerate(train):
#     if t['class_true'] == 2:
#         print(i)

In [6]:
model = BaseModel(train)
model.fit()

2022-11-20 20:14:27.121476: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-20 20:14:27.121976: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 200, 2)            0         
_________________________________________________________________
conv1d (Conv1D)              (None, 189, 16)           400       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 178, 16)           3088      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 59, 16)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 48, 32)            6176      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 37, 32)            12320     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0

2022-11-20 20:14:29.205567: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/5
2151/2151 [==============================] - 140s 65ms/step - loss: 0.3409 - accuracy: 0.8565 - val_loss: 0.3580 - val_accuracy: 0.8269
Epoch 2/5
2151/2151 [==============================] - 137s 64ms/step - loss: 0.2636 - accuracy: 0.8732 - val_loss: 0.2511 - val_accuracy: 0.8983
Epoch 3/5
2151/2151 [==============================] - 136s 63ms/step - loss: 0.1343 - accuracy: 0.9452 - val_loss: 0.2542 - val_accuracy: 0.9138
Epoch 4/5
2151/2151 [==============================] - 136s 63ms/step - loss: 0.0930 - accuracy: 0.9616 - val_loss: 0.2551 - val_accuracy: 0.9141
Epoch 5/5
2151/2151 [==============================] - 137s 64ms/step - loss: 0.0762 - accuracy: 0.9680 - val_loss: 0.3205 - val_accuracy: 0.9120
